# YOLOv8 Exercise

The contents of this jupyter notebook is my exploration on YOLOv8, following the [YOLOv8 Tutorial](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb) notebook.  


# 0. Preparation

## 0-1. Install `ultralytics`

First, install `ultralytics` with pip.  
Then check if `ultralytics` was correctly installed.  

In [ ]:
%pip install ultralytics # install ultralytics
import ultralytics
ultralytics.checks() # check if ultralytics is installed correctly

## 0-2. Prepare images

Since my goal is to detect human faces, I prepared some free images containing human faces.  
The format of the images are fixed as `jpeg`, and are named like `human_x.jpeg` where `x` is the image index(no special meaning in the index).   

The images are downloaded from [here](https://www.pexels.com/search/human%20face/).  
They are saved right under `img` directory.

# 1. Use a pretrained model

Mainly two ways are provided to use YOLOv8.

1. A command line tool
2. A python API

I chose to use the python.

## 1-1. Detect a person

Using the pretrained detection model "yolov8n.pt", detection of a person was done.

In [ ]:
from ultralytics import YOLO

# load a pretrained model (recommended for training)
model = YOLO("yolov8n.pt") 

# results = model("img/human_1.jpeg")
results = model.predict(source="img/human_1.jpeg", show=True) # same as above

print(results)

## 1-2. Detect face

I downloaded the pretrained face detection model("yolov8n-face.pt") from [here](https://github.com/akanametov/yolov8-face?tab=readme-ov-file)  
Using the model, I got the (x,y,width,height) of the face ROI.

In [ ]:
# load a pretrained model (recommended for training)
model = YOLO("./yolov8n-face.pt") 

# results = model("img/human_1.jpeg")
results = model.predict(source="img/human_2.jpeg", show=True, save=True)
# results = model.predict(source="img/chara_1.png", show=True, save=True)


# print(results)
# print(results[0].boxes[0].xyxy)
# print(results[0].boxes[0].xywh)
human_box_xywh = results[0].boxes[0].xywh[0]
print("x: ", results[0].boxes[0].xywh[0][0])
print("y: ", results[0].boxes[0].xywh[0][1])
print("w: ", results[0].boxes[0].xywh[0][2])
print("h: ", results[0].boxes[0].xywh[0][3])

## 1-3. Detect animation character face

It is a bit questionable whether detecting a character's face with the same model is approperiate.  
At the same time, using the same model can be a good approach because detection standard is the same.  
For example if the sizes of two different face ROIs are simililer, the face sizes could be thought as similier(just my assumption).  

In [ ]:
# load a pretrained model (recommended for training)
model = YOLO("./yolov8n-face.pt") 

# results = model("img/human_1.jpeg")
# results = model.predict(source="img/human_2.jpeg", show=True, save=True)
results = model.predict(source="img/chara_1.png", show=True)


# print(results)
# print(results[0].boxes[0].xyxy)
# print(results[0].boxes[0].xywh)
chara_box_xywh = results[0].boxes[0].xywh[0]
print("x: ", results[0].boxes[0].xywh[0][0])
print("y: ", results[0].boxes[0].xywh[0][1])
print("w: ", results[0].boxes[0].xywh[0][2])
print("h: ", results[0].boxes[0].xywh[0][3])

# 2. Placing an image of a character according to the face detection

The final goal of this notebook is to resize a prepared image of a character and placing it alongside the detected face.

## 2-1. Adjusting image size.

By comparing `human_box_xywh` and `chara_box_xywh`, resizing the character image to make its size similer to the human might be possible.  
I tried three types of resizing.  

1. adjust to width
2. adjust to height
3. adjust to area(width * height)

In [ ]:
from ultralytics import YOLO

# load a pretrained model (recommended for training)
model = YOLO("./yolov8n-face.pt")

results = model.predict(source="img/human_2.jpeg")
human_box_xywh = results[0].boxes[0].xywh[0]

results = model.predict(source="img/chara_1.png")
chara_box_xywh = results[0].boxes[0].xywh[0]

"""
Human image size is fixed.(Only character image size is adjusted to human image size.)
"""
from PIL import Image
import math

chara_image = Image.open("img/chara_1.png")

# 1. adjust to width
new_width = int(chara_image.width * human_box_xywh[2] / chara_box_xywh[2])
new_height = int(chara_image.height * human_box_xywh[2] / chara_box_xywh[2])
resized_chara_image = chara_image.resize((new_width, new_height))
resized_chara_image.save("resized_img/resized_by_width_chara_1.png")

# # 2. adjust to height
new_width = int(chara_image.width * human_box_xywh[3] / chara_box_xywh[3])
new_height = int(chara_image.height * human_box_xywh[3] / chara_box_xywh[3])
resized_chara_image = chara_image.resize((new_width, new_height))
resized_chara_image.save("resized_img/resized_by_height_chara_1.png")

# # 3. adjust to area(width * height)

new_width = int(chara_image.width * math.sqrt((human_box_xywh[2]*human_box_xywh[3]) / (chara_box_xywh[2]*chara_box_xywh[3])))
new_height = int(chara_image.height * math.sqrt((human_box_xywh[2]*human_box_xywh[3]) / (chara_box_xywh[2]*chara_box_xywh[3])))
resized_chara_image = chara_image.resize((new_width, new_height))
resized_chara_image.save("resized_img/resized_by_area_chara_1.png")

## 2-2. Draw(?) the character image over ther human image

Just tried for the case of "1. adjust to width".

In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFilter

sub_img_names = [
    "resized_by_width_chara_1.png",
    "resized_by_height_chara_1.png",
    "resized_by_area_chara_1.png"
]

model = YOLO("./yolov8n-face.pt")
for sub_img_name in sub_img_names:
    base_img = Image.open("img/human_2.jpeg")
    sub_img = Image.open(f"resized_img/{sub_img_name}")

    results = model.predict(source="img/human_2.jpeg")
    h_xywh = results[0].boxes[0].xywh[0]
    # (h_x, h_y, h_w, h_h) = results[0].boxes[0].xywh[0]

    results = model.predict(source=f"resized_img/{sub_img_name}")
    c_xywh = results[0].boxes[0].xywh[0]
    # (c_x, c_y, c_w, c_h) = results[0].boxes[0].xywh[0]

    margin = 50
    base_img.paste(sub_img,
                    box=(int(h_xywh[0] + h_xywh[2] + margin - c_xywh[0]),
                            int(h_xywh[1] + h_xywh[3]/2 - c_xywh[1] - c_xywh[3]/2))
                    ,mask=sub_img)
    base_img.save(f"regenerated_img/{sub_img_name}")

# # image paste on case of "1. adjust to width"
# base_img = Image.open("img/human_2.jpeg")
# sub_img = Image.open("resized_img/resized_by_area_chara_1.png")

# model = YOLO("./yolov8n-face.pt")
# results = model.predict(source="img/human_2.jpeg")
# h_xywh = results[0].boxes[0].xywh[0]
# results = model.predict(source="resized_img/resized_by_area_chara_1.png")
# c_xywh = results[0].boxes[0].xywh[0]

# # paste image
# margin = 50 * human_box_xywh[2] / chara_box_xywh[2]
# base_img.paste(sub_img,
#                box=(int(h_xywh[0] + h_xywh[2] + margin - c_xywh[0]),
#                 int(h_xywh[1] + h_xywh[3]/2 - c_xywh[1] - c_xywh[3]/2))
#                 ,mask=sub_img)
# base_img.show()

# base_img.save("regenerated_img/regenerated_by_weight_chara_1.png")

In [ ]:
model.predict(source="regenerated_img/resized_by_width_chara_1.png", save=True, show=True)